In [ ]:
import os
import pandas as pd

In [ ]:
data_dir = "/".join(os.getcwd().split("/")[:-1] + ['Mpileup'])

files = [f for f in os.listdir(data_dir) if f.endswith('headerless.vcf')]

# Instantiate base dataframe to merge into

In [ ]:
f = files[0]
f_path = data_dir + "/" + f
f_name = f.split(".")[0]
header = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", f_name]

temp = pd.read_csv(f_path, sep="\t", header=None, names=header)
to_keep = ["CHROM", "POS", f_name]
temp = temp[to_keep]

# Only want reference vs nonreference, and depth

format is GT:DP:RO:QR:AO:QA:GL
only care about:
- DP ~ depth
- RO ~ reference counts
- AO ~ alternate counts

In [ ]:
def ExtractValues(value_string):
    return sum(map(int,value_string.split(",")))
    
temp[f_name+"_depth"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[1]))
temp[f_name+"_ref"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[2]))
temp[f_name+"_alt"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[4]))
temp[f_name+"_AF"] = temp[f_name+"_ref"] / temp[f_name+"_depth"]
temp = temp.drop(f_name, axis=1)
data = temp

# Merge all other files into one dataframe

In [ ]:
for f in files[1:]:
    f_path = data_dir + "/" + f
    f_name = f.split(".")[0]
    header = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", f_name]

    temp = pd.read_csv(f_path, sep="\t", header=None, names=header)
    to_keep = ["CHROM", "POS", f_name]
    temp = temp[to_keep]
    
    temp[f_name+"_depth"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[1]))
    temp[f_name+"_ref"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[2]))
    temp[f_name+"_alt"] = temp[f_name].apply(lambda row: ExtractValues(row.split(":")[4]))
    temp[f_name+"_AF"] = temp[f_name+"_ref"] / temp[f_name+"_depth"]
    temp = temp.drop(f_name, axis=1)
    data = pd.merge(data, temp, on=["CHROM", "POS"], how='outer')

In [ ]:
save_path = data_dir + "/" "editing_levels.tsv"
# save_path = "/Users/Cameron/editing_levels.tsv"
data.to_csv(path_or_buf=save_path, sep="\t", index=False)